## 7.5 NotyApp

Aplikacja będzie zawierać prostą listę notatek/zadań zapisaną w bazie danych `SQLite`. Lista będzie dostępna z poziomu `Widget` jak i w samej aplikacji. Przyjrzymy się jak zaimplemntować podstawowe elementy komunikacji między widgetem a naszą aplikacją, oraz jak aktualizować dane wyświetlane w `ListView` na `Widget`.

### **Layout**

Rozpocznijmy od utworzenia layoutów dla samego widgetu oraz dla pojedynczego elementu `ListView` (podobnie jak w `RecyclerView`). Layout widgetu będzie składał się z samego `ListView` oraz pola `TextView`, które będzie wyświetlane gdy lista jest pusta

In [ ]:
// noty_widget_provider.xml
<RelativeLayout xmlns:android="http://schemas.android.com/apk/res/android"
    style="@style/Widget.NotyJava.AppWidget.Container"
    android:layout_width="match_parent"
    android:layout_height="match_parent"
    android:theme="@style/Theme.NotyJava.AppWidgetContainer">

    <ListView
        android:id="@+id/listViewWidget"
        android:layout_width="match_parent"
        android:layout_height="match_parent"/>

    <TextView
        android:id="@+id/emptyViewTextView"
        android:layout_width="match_parent"
        android:layout_height="match_parent"
        android:textSize="20sp"
        android:text="lista jest pusta"/>
</RelativeLayout>

Następnie dodajmy layout pojedynczego elementu listy - wstępnie będzie to tylko pole `TextView`

In [ ]:
// item_view.xml
<RelativeLayout xmlns:android="http://schemas.android.com/apk/res/android"
    android:layout_width="match_parent"
    android:layout_height="wrap_content">

    <TextView
        android:id="@+id/itemListTextView"
        android:layout_width="match_parent"
        android:layout_height="110dp"
        android:gravity="center"
        android:text="przykład długiego tekstu"
        android:background="@color/teal_200"
        android:textColor="@color/black"/>

</RelativeLayout>

Dodajmy opis widgetu do katalogu `xml`

In [ ]:
// noty_widget_provider_info.xml
<appwidget-provider xmlns:android="http://schemas.android.com/apk/res/android"
    android:initialLayout="@layout/noty_widget_provider"
    android:minHeight="110dp"
    android:minResizeHeight="40dp"
    android:minWidth="110dp"
    android:resizeMode="vertical|horizontal"
    android:updatePeriodMillis="36000000"
    android:widgetCategory="home_screen" />

Wstępnie dane będziemy pobierać (jak zwykle) z `DataProvider`

In [ ]:
object DataProvider {
    val dummyData = listOf(
        "notatka 1", 
        "notatka 2", 
        "notatka 3", 
        "notatka 4", 
        "notatka 5", 
        "notatka 6", 
        "notatka 7", 
        "notatka 8", 
        "notatka 9")
}

### **RemoteViewsService**

Rozpoczniemy od implementacji odpowiedniej usługi - jest ona niezbędna ponieważ widget działa na kompletnie innym procesie i nie możemy się komunikować wprost. Klasa `RemoteViewsService` jest usługą, z którą widgety będą się łączyć aby adapter mógł otrzymać instancje `RemoteViews`

In [ ]:
class NotyWidgetService : RemoteViewsService() {
    override fun onGetViewFactory(intent: Intent): RemoteViewsFactory {
        return null
    }

Musimy zaimplementować jedną metodę `onGetViewFactory`, która zwraca obiekt dostarczający dane. W tym celu zaimplementujemy klasę implementującą interfejs `RemoteViewsFactory` zawierającą szerego niezbędnych metod
- `onCreate` - tutaj łączymy się z bazą danych, wywoływana zaraz po konstruktorze
- `onDataSetChanged` - wywoływana gdy adapter wywołuje `notifyDataSetChanged`
- `onDestroy` - wywoływana w momencie usunięcia powiązania z ostatnim adapterem
- `getCount` - zwraca liczbę elementów w kolekcji
- `getViewAt` - zwraca obiekt `View` powiązany z daną pozycją
- `getLoadingView` - zezwala na załączenie własnego `View` dla ekranu ładowania
- `getViewTypeCount` - zwraca liczbę **różnych** typów `View` wykorzystywanych przez adapter - w tym przykładzie będziemy wykorzystywać jeden typ
- `getItemId` - zwraca identyfikator - tutaj będzie to odpowiednikiem rzędu na liście
- `hasStableIds` - określa czy identyfikatory pozostają niezmienne przy zmianie powiązanych z nim danych

In [ ]:
class NotyWidgetItemFactory() : RemoteViewsFactory {
    override fun onCreate() {}

    override fun onDataSetChanged() {}
    override fun onDestroy() {}

    override fun getCount(): Int {}

    override fun getViewAt(position: Int): RemoteViews {}

    override fun getLoadingView(): RemoteViews? {}

    override fun getViewTypeCount(): Int {}

    override fun getItemId(position: Int): Long {}

    override fun hasStableIds(): Boolean {}
}

Rozpocznijmy od dodania i inicjalizacji kilku pól

In [ ]:
class NotyWidgetItemFactory(private val context: Context, intent: Intent) :
    RemoteViewsFactory {
    private val appWidgetId: Int
    private lateinit var noteList: List<String>
        
    init {
            appWidgetId = intent.getIntExtra(
                AppWidgetManager.EXTRA_APPWIDGET_ID,
                AppWidgetManager.INVALID_APPWIDGET_ID)
        }  
    }

- W metodzie `onCreate` zainicjujemy listę notatek - w pierwszej fazie będzie to lista z klasy `DataProvider`

In [ ]:
override fun onCreate() {
    // otworz baze danych
    noteList = DataProvider.dummyData
}

- `getCount` zwraca wielkość listy

In [ ]:
override fun getCount(): Int = noteList.size

- W tym przykładzie będziemy wykorzystywać z domyślnego widoku ładowania, więc metoda zwraca `null`

In [ ]:
override fun getLoadingView(): RemoteViews? {
    return null
}

- Posiadamy jeden typ `View`, więc zwracamy `1`

In [ ]:
override fun getViewTypeCount(): Int = 1

- ponieważ w pierwszej fazie będziemy korzystać tylko z listy, posłużymy się pozycją jako identyfikatorem

In [ ]:
override fun getItemId(position: Int): Long = position.toLong()

- nasza kolekcja w tej chwili posiada stabilne `id`

In [ ]:
override fun hasStableIds(): Boolean = true

- w metodzie `getViewAt` zwracamy odpowiedni obiekt

In [ ]:
override fun getViewAt(position: Int): RemoteViews {
    val remoteViews = RemoteViews(context.packageName, R.layout.item_list)
    remoteViews.setTextViewText(R.id.itemListTextView, noteList[position])
    return remoteViews
}

Powracamy do klasy `NotyWidgetService` i w metodzie `onGetViewFactory` zwracamy instancję `NotyWidgetItemFactory`

In [ ]:
class NotyWidgetService : RemoteViewsService() {
    override fun onGetViewFactory(intent: Intent): RemoteViewsFactory {
        return NotyWidgetItemFactory(applicationContext, intent)
    }
    ...
}

### **AppWidgetProvider**

Drugim niezbędnym elementem będzie `AppWidgetProvider`

In [ ]:
class NotyWidgetProvider : AppWidgetProvider() {
    override fun onUpdate(
        context: Context,
        appWidgetManager: AppWidgetManager,
        appWidgetIds: IntArray
    ) {}

W metodzie `onUpdate` przechodzimy przez wszystkie instancje naszego widgetu

In [ ]:
for (appWidgetId in appWidgetIds) {

Tworzymy nowy `Intent` przekierowujący do usługi

In [ ]:
    val serviceIntent = Intent(context, NotyWidgetService::class.java)

następnie dodajmy `id` widgetu oraz wykorzystujemy metodę `setData` - metoda ta wskazuje lokalizację obiektu (przykładowo może być to plik)

In [ ]:
    serviceIntent.apply {
        putExtra(AppWidgetManager.EXTRA_APPWIDGET_ID, appWidgetId)
        data = Uri.parse(serviceIntent.toUri(Intent.URI_INTENT_SCHEME))
    }

Tworzymy nasze `RemoteViews`

In [ ]:
    val views = RemoteViews(context.packageName, R.layout.noty_widget_provider)

ustawiamy adapter i widok dla pustej kolekcji

In [ ]:
    views.apply {
        setRemoteAdapter(R.id.listViewWidget, serviceIntent)
        setEmptyView(R.id.listViewWidget, R.id.emptyViewTextView)
    }

na koniec wywołujemy metodę `updateAppWidget`

In [ ]:
    appWidgetManager.updateAppWidget(appWidgetId, views)

Po wyjściu z pętli `for` wywołujemy metodę superklasy

In [ ]:
    }
super.onUpdate(context, appWidgetManager, appWidgetIds)

Pełny kod klasy `NotyWidgetProvider`

In [ ]:
class NotyWidgetProvider : AppWidgetProvider() {
    override fun onUpdate(
        context: Context,
        appWidgetManager: AppWidgetManager,
        appWidgetIds: IntArray
    ) {
        for (appWidgetId in appWidgetIds) {
            val serviceIntent = Intent(context, NotyWidgetService::class.java)
            serviceIntent.apply {
                putExtra(AppWidgetManager.EXTRA_APPWIDGET_ID, appWidgetId)
                data = Uri.parse(serviceIntent.toUri(Intent.URI_INTENT_SCHEME))
            }
            val views = RemoteViews(context.packageName, R.layout.noty_widget_provider)
            views.apply {
                setRemoteAdapter(R.id.listViewWidget, serviceIntent)
                setEmptyView(R.id.listViewWidget, R.id.emptyViewTextView)
            }
            appWidgetManager.updateAppWidget(appWidgetId, views)
        }
        super.onUpdate(context, appWidgetManager, appWidgetIds)
    }
}

Do `AndroidManifest` musimy wprowadzić informacje o naszym providerze oraz service.

In [ ]:
<receiver
    android:name=".provider.NotyWidgetProvider"
    android:exported="false">
    <intent-filter>
        <action android:name="android.appwidget.action.APPWIDGET_UPDATE" />
    </intent-filter>

    <meta-data
        android:name="android.appwidget.provider"
        android:resource="@xml/noty_widget_provider_info" />
</receiver>

<service
    android:name=".service.NotyWidgetService"
    android:permission="android.permission.BIND_REMOTEVIEWS" />

Możemy przetestować aplikację i widget.

<table><tr><td><img src="https://media2.giphy.com/media/U0lFKx7L0xtZHrpaLQ/giphy.gif?cid=790b76119fc11ce6d16c4ba7009680f1447b35e164c8012a&rid=giphy.gif&ct=g" width="150" /></td><td><img src="https://media2.giphy.com/media/1BNcJ99feU4EosbU19/giphy.gif?cid=790b76117e6f80ef480e50e651e2e981d067cfeb27c6e32c&rid=giphy.gif&ct=g" width="150" /></td</tr></table>